# Lab 16 Assignment 3 - Group Assignment

When creating ML models, the concept of efficiency has three sides:
1. The time dedicated by the analyst to build the model
2. The computer time and resources needed by the final model
3. The accuracy of the final model

Efficiency is a combination of all

In this assignment, you are asked to be efficient. Spark is the best tool to build models over massive datasets

If you need to create Spark+Python Machine Learning models that "run fast" on the  cluster, you must avoid using Python code or working with RRD+python. Try to use  the already existing methods that do what you need (do not reinvent the wheel).

Therefore try to use the implemented object+methods inside the Spark SQL and ML modules. They are very fast, because it is compiled Java/Scala code. Try to use: DataFrames, Feature Transfomers, Estimators, Pipelines, GridSearch, CV, ...

For this assignment, you are asked to create a classification model that:
1. Uses the variables in the dataset (train.csv) to predict label "loan_status"
2. Write a python scripts that:
    - Reads the "train.csv" and "test.csv" files, transform and select variables as you wish.
    - Train/fit your model using the "train.csv".
    - Predict your model on the "test.csv" ( you should generate a file with your predictions).
    - I will use a different test dataset (with the true loan_status).

Your work will be evaluated under the following scoring schema
- (40%) ETL process
- (40%) Model train process
- (10%) Code Readability 
- (10%) AUC on the test set (at least 50%)

Enjoy it and best of luck!!


This Assignment is based on kaggle competition https://www.kaggle.com/c/loan-default-prediction from where a sub-dataset has been taken.

### File descriptions
**train.csv** - the training set (to use for building a model)

**test.csv** - the test set (to use for applying predictings)

**sample_submission.csv** - a template for the submission file

### Data Description (also contained in LendingClub_DataDescription.csv)
**ID**: A unique LC assigned ID for the loan listing.

**loan_amnt**: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

**loan_status**: Current status of the loan (**Target**: 1 = Charged Off, 0 = Fully Paid).

**term**: The number of payments on the loan. Values are in months and can be either 36 or 60.

**int_rate**: Interest Rate on the loan.

**installment**: The monthly payment owed by the borrower if the loan originates.

**emp_length**: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.

**home_ownership**: The home ownership status provided by the borrower during registration. Our values are: OTHER/NONE, MORTGAGE, OWN, RENT.

**annual_inc**: The self-reported annual income provided by the borrower during registration.

**purpose**: A category provided by the borrower for the loan request.

**title**: The loan title provided by the borrower.

**STATE**: The state provided by the borrower in the loan application.

**delinq_2yrs**: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years.

**revol_bal**: Total credit revolving balance.  revolve a balance -- that is, not pay it off at the end of the month 

**revol_util**: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

**total_pymnt**: Indicates total payment at the end of the loan.

In [1]:
import os
import sys

os.environ['SPARK_HOME'] = "C:\\spark-2.3.2-bin-hadoop2.7\\"

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exist. The names might change
#as versions change.
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.7-src.zip"))

#Initialize SparkSession and SparkContext
from pyspark.sql import SparkSession

#Create a Spark Session
spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("MiPrimer") \
    .config("spark.executor.memory", "6g") \
    .config("spark.cores.max","4") \
    .getOrCreate()


#Get the Spark Context from Spark Session    
sc = spark.sparkContext

In [2]:
import numpy as np
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql import functions as f
from pyspark.ml.feature import Normalizer, Imputer, StringIndexer, OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.param import Params
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel, RandomForestClassifier,GBTClassifier

### 1. Read the files into a Dataframes and show them

In [3]:
loanDF = spark.read.format('csv') \
           .option("inferSchema", "true") \
           .option("header","true")\
           .option("delimiter", ";") \
           .load('../data/train.csv') 

testDF = spark.read.format('csv') \
            .option("inferSchema", "true") \
            .option("header","true")\
            .option("delimiter", ";") \
            .load('../data/test.csv') 

### 2. Analizing null values, cross tables distribution and covariances

#### 2.1. Null values

In [4]:
loanDF_empty=loanDF.select([f.count(f.when(col(c).isNull() | f.isnan(c), c)).alias(c) for c in loanDF.columns])
print('Total number of rows = {0}. Total number of missing values, per column:'.format(loanDF.count()))
loanDF_empty.toPandas()

Total number of rows = 29755. Total number of missing values, per column:


,ID,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,purpose,title,STATE,delinq_2yrs,revol_bal,revol_util,total_pymnt,loan_status
0,0,0,0,0,0,0,0,0,1229,1238,1239,1239,1239,1275,1239,0


#### 2.2. Cross table distribution

In [5]:
cross_list=[x for x in loanDF.columns if x in ['term','emp_length','home_ownership','STATE']]
[loanDF.crosstab(c,'loan_status').show() for c in cross_list]

+----------------+-----+----+
|term_loan_status|    0|   1|
+----------------+-----+----+
|       60 months| 6203|1846|
|       36 months|19327|2379|
+----------------+-----+----+

+----------------------+----+----+
|emp_length_loan_status|   0|   1|
+----------------------+----+----+
|               5 years|2149| 348|
|               6 years|1435| 235|
|               3 years|2675| 387|
|               8 years| 950| 160|
|                1 year|2097| 343|
|              < 1 year|2968| 478|
|               9 years| 820| 120|
|                   n/a| 622| 170|
|               4 years|2245| 346|
|             10+ years|5599|1006|
|               7 years|1127| 205|
|               2 years|2843| 427|
+----------------------+----+----+

+--------------------------+-----+----+
|home_ownership_loan_status|    0|   1|
+--------------------------+-----+----+
|                     OTHER|   60|   9|
|                       OWN| 1950| 322|
|                  MORTGAGE|11514|1774|
|                 

[None, None, None, None]

#### ETL summary

You should explain how you are going to clean and prepare the data  
1. Deal with data types  
2. Create ratio  
3. Drop fiels  
4. Impute numerical missing values with median
5. Normalize numerical values  
6. Convert categories into index  
7. Convert index into onehotencoding


### 2.1 Spark code about the one explained in "ETL Summary"

In [6]:
#OWN CONSTRUCTED FUNCTIONS FOR PIPELINE

#data types
def fix_data_types(data):
    data=data.withColumn('int_rate', col('int_rate').substr(1, 4))\
             .withColumn("int_rate", col("int_rate").cast('float'))\
             .withColumn('revol_util', col('revol_util').substr(1, 4))\
             .withColumn("revol_util", col("revol_util").cast('float'))\
             .withColumn("delinq_2yrs", col("delinq_2yrs").cast('float'))\
             .withColumn("revol_bal", col("revol_bal").cast('float'))
    return data

def create_ratio(data):
    data=data.withColumn('r_ins_minc',(data['installment']/(data['annual_inc']/12)))
    return data

def drop_fields(data):
    data=data.drop('title','home_ownership')
    return data

def update_df(data, operations):
    for ops in operations:
        data=ops(data)
    return data    

### 2.2 Spark code about the transformations you apply to the variables

In [7]:
#ml lib INTEGRATED FUNCTIONS FOR PIPELINE
stages=[]

#Missings values

empty_values=["delinq_2yrs","revol_bal","revol_util","total_pymnt"]
null_imputer=Imputer()\
        .setStrategy('median')\
        .setInputCols(empty_values)\
        .setOutputCols(empty_values)\


stages.append(null_imputer)

#Normalize
# normalize_list=['revol_bal','total_pymnt','annual_inc']
# for val in normalize_list:
#     normal=Normalizer(inputCol=val,outputCol=val+'_nor')
#     stages.append(normal)

#Convert categories to numbers
convert_list=['term','emp_length','STATE','purpose','loan_status']
for val in convert_list:
    if val=='loan_status':
        string_to_index = StringIndexer(inputCol = val, outputCol = val+'_index')
        stages.append(string_to_index)
    else:
        string_to_index = StringIndexer(inputCol = val, outputCol = val+'_index',handleInvalid='keep')
        stages.append(string_to_index)
    
#One hot encoding
encoder = OneHotEncoderEstimator(inputCols=["term_index",'emp_length_index','purpose_index','STATE_index'],
                                           outputCols=["term_enc",'emp_length_enc','purpose_enc','STATE_enc'])
stages.append(encoder)

### 2.3 Code to assemble the variables to a numerical vector (VectorAssembler)

In [8]:
assembler = VectorAssembler(inputCols=['loan_amnt','installment', 'annual_inc', 'delinq_2yrs', 'revol_bal', 'revol_util',\
                                        'total_pymnt','r_ins_minc','emp_length_enc','term_enc','purpose_enc','STATE_enc'],\
                            outputCol='features')
stages.append(assembler)

#### Make sure stages contain all required steps

In [9]:
stages

[Imputer_4ee899a81de7a97114f0,
 StringIndexer_46478ca9cc9765aef391,
 StringIndexer_4c83b34edf7498ddaff3,
 StringIndexer_40719376c3b529b0ce7b,
 StringIndexer_4c6a84ef73831566b8fd,
 StringIndexer_478dbbbe5935851d78d0,
 OneHotEncoderEstimator_4fbcaa2a9f91cbcf07a5,
 VectorAssembler_423b846c9de2041b869e]

### 2.4 Create Pipeline for train and test data

In [10]:
preprocessing=[fix_data_types,create_ratio,drop_fields]
pipeline=Pipeline(stages=stages)

trainDF=update_df(loanDF,preprocessing)
model=pipeline.fit(trainDF)
trainDF1=model.transform(trainDF)

testDF=update_df(testDF,preprocessing)
testDF1=model.transform(testDF)

After all the steps applied in previous box, the **train and test dataset are cleaned and prepared** so that the models can be applied on them.

In [11]:
trainDF1.toPandas().head(10)

,ID,loan_amnt,term,int_rate,installment,emp_length,annual_inc,purpose,STATE,delinq_2yrs,...,term_index,emp_length_index,STATE_index,purpose_index,loan_status_index,term_enc,emp_length_enc,purpose_enc,STATE_enc,features
0,2,2500,60 months,15.20,59.83,< 1 year,30000.0,car,GA,0.0,...,1.0,1.0,7.0,6.0,1.0,"(0.0, 1.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(2500.0, 59.83, 30000.0, 0.0, 1687.0, 0.090000..."
1,4,10000,36 months,13.40,339.31,10+ years,49200.0,other,CA,0.0,...,0.0,0.0,0.0,2.0,0.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(10000.0, 339.31, 49200.0, 0.0, 5598.0, 0.2099..."
2,5,3000,60 months,12.60,67.79,1 year,80000.0,other,OR,0.0,...,1.0,6.0,24.0,2.0,0.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3000.0, 67.79, 80000.0, 0.0, 27783.0, 0.52999..."
3,6,5000,36 months,7.90,156.46,3 years,36000.0,wedding,AZ,0.0,...,0.0,3.0,12.0,7.0,0.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5000.0, 156.46, 36000.0, 0.0, 7963.0, 0.28000..."
4,7,7000,60 months,15.90,170.08,8 years,47004.0,debt_consolidation,NC,0.0,...,1.0,9.0,15.0,0.0,0.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(7000.0, 170.08, 47004.0, 0.0, 17726.0, 0.8500..."
5,8,3000,36 months,18.60,109.43,9 years,48000.0,car,CA,0.0,...,0.0,10.0,0.0,6.0,0.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3000.0, 109.43, 48000.0, 0.0, 8221.0, 0.87000..."
6,10,5375,60 months,12.60,121.45,< 1 year,15000.0,other,TX,0.0,...,1.0,1.0,3.0,2.0,1.0,"(0.0, 1.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5375.0, 121.45, 15000.0, 0.0, 9279.0, 0.36000..."
7,11,6500,60 months,14.60,153.45,5 years,72000.0,debt_consolidation,AZ,0.0,...,1.0,5.0,12.0,0.0,0.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(6500.0, 153.45, 72000.0, 0.0, 4032.0, 0.20000..."
8,12,12000,36 months,12.60,402.54,10+ years,75000.0,debt_consolidation,CA,0.0,...,0.0,0.0,0.0,0.0,0.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(12000.0, 402.54, 75000.0, 0.0, 23336.0, 0.670..."
9,14,3000,36 months,9.91,96.68,3 years,15000.0,credit_card,IL,0.0,...,0.0,3.0,5.0,1.0,0.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(3000.0, 96.68, 15000.0, 0.0, 7323.0, 0.430000..."


In [12]:
testDF1.toPandas().head(10)

,ID,loan_amnt,term,int_rate,installment,emp_length,annual_inc,purpose,STATE,delinq_2yrs,...,term_index,emp_length_index,STATE_index,purpose_index,loan_status_index,term_enc,emp_length_enc,purpose_enc,STATE_enc,features
0,1,5000,36 months,10.600000,162.87,10+ years,24000.0,credit_card,AZ,0.0,...,0.0,0.0,12.0,1.0,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5000.0, 162.87, 24000.0, 0.0, 13648.0, 0.8399..."
1,3,2400,36 months,15.900000,84.33,10+ years,12252.0,small_business,IL,0.0,...,0.0,0.0,5.0,5.0,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(2400.0, 84.33, 12252.0, 0.0, 2956.0, 0.990000..."
2,9,5600,60 months,21.200001,152.39,4 years,40000.0,small_business,CA,0.0,...,1.0,4.0,0.0,5.0,1.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5600.0, 152.39, 40000.0, 0.0, 5210.0, 0.33000..."
3,13,9000,36 months,13.400000,305.38,< 1 year,30000.0,debt_consolidation,VA,0.0,...,0.0,1.0,8.0,0.0,1.0,"(1.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(9000.0, 305.38, 30000.0, 0.0, 10452.0, 0.9200..."
4,15,10000,36 months,10.600000,325.74,3 years,100000.0,other,CA,0.0,...,0.0,3.0,0.0,2.0,1.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(10000.0, 325.74, 100000.0, 0.0, 11997.0, 0.56..."
5,18,3600,36 months,6.030000,109.57,10+ years,110000.0,major_purchase,CT,0.0,...,0.0,0.0,16.0,4.0,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3600.0, 109.57, 110000.0, 0.0, 22836.0, 0.159..."
6,26,15000,36 months,9.910000,483.38,2 years,92000.0,credit_card,IL,0.0,...,0.0,2.0,5.0,1.0,1.0,"(1.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(15000.0, 483.38, 92000.0, 0.0, 13707.0, 0.939..."
7,27,15000,36 months,14.200000,514.64,9 years,60000.0,debt_consolidation,NY,0.0,...,0.0,10.0,1.0,0.0,1.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(15000.0, 514.64, 60000.0, 0.0, 5872.0, 0.5799..."
8,29,4000,36 months,11.700000,132.31,10+ years,106000.0,debt_consolidation,FL,1.0,...,0.0,0.0,2.0,0.0,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(4000.0, 132.31, 106000.0, 1.0, 6110.0, 0.3799..."
9,31,4375,36 months,7.510000,136.11,7 years,17108.0,debt_consolidation,NY,0.0,...,0.0,8.0,1.0,0.0,1.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(4375.0, 136.11, 17108.0, 0.0, 11210.0, 0.8700..."


#### Split train set into training and validation

In [13]:
(trainingData, validationData) = trainDF1.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(validationData.count())

20882
8873


### 3. Logistic Regresion Model

### 3.1 Write a function "metrics" which has a LogisticRegressionModel.summary as input attribute and produces an output of: 
1. Area under ROC
2. False Positive Rate By Label
3. True Positive Rate By Label
4. Precision By Label
5. Recall By Label
6. fMeasure By Label
7. Accuracy
8. False Positive Rate
9. True Positive Rate
10. fMeasure
11. Precision
12. Recall

In [14]:
#The fitted model from a Pipeline is a PipelineModel
def metrics(trainingSummary):
    '''This function takes a model that has been fitted (logisticregressionModel) and extracts all metrics'''
    print("1. Area under ROC: "+str(trainingSummary.areaUnderROC))
    print("2. False Positive Rate by Label: "+str(trainingSummary.falsePositiveRateByLabel))
    print("3. True Positive Rate by Label: "+str(trainingSummary.truePositiveRateByLabel))
    print("4. Precision by Label: "+str(trainingSummary.precisionByLabel))
    print("5. Recall by Label: "+str(trainingSummary.recallByLabel))
    print("6. F measure by Label: "+str(trainingSummary.fMeasureByLabel))
    print("7. Accuracy: "+str(trainingSummary.accuracy))
    print("8. False Positive Rate: "+str(trainingSummary.weightedFalsePositiveRate))
    print("9. True Positive Rate: "+str(trainingSummary.weightedTruePositiveRate))
    print("10. F measure: "+str(trainingSummary.weightedFMeasure))
    print("11. Precision: "+str(trainingSummary.weightedPrecision))
    print("12. Recall: "+str(trainingSummary.weightedRecall))

### 3.2 Apply a Logistic Regresion Base Model and show the metrics by the function above

In [15]:
lr = LogisticRegression() \
        .setFeaturesCol("features") \
        .setLabelCol("loan_status_index")

lr_model=lr.fit(trainingData)
#print(lr_model.explainParams())
metrics(lr_model.summary)

1. Area under ROC: 0.9211250801809421
2. False Positive Rate by Label: [0.38058705803869247, 0.008051889957503914]
3. True Positive Rate by Label: [0.9919481100424961, 0.6194129419613076]
4. Precision by Label: [0.9395688787670144, 0.9280359820089955]
5. Recall by Label: [0.9919481100424961, 0.6194129419613076]
6. F measure by Label: <bound method LogisticRegressionSummary.fMeasureByLabel of <pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x0000029F72C3AFD0>>
7. Accuracy: 0.9384637486830764
8. False Positive Rate: 0.3271026966792727
9. True Positive Rate: 0.9384637486830764
10. F measure: <bound method LogisticRegressionSummary.weightedFMeasure of <pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x0000029F72C3AFD0>>
11. Precision: 0.9379131166522485
12. Recall: 0.9384637486830764


### 3.2 We are going to try to improve our model:
1. Using a `weight column` in our Logistic Regression Model (Take into account we are working with a unbalanced dataset)
2. Define a `ParamGridBuilder` with `regParam`, `elasticNetParam` and `maxIter` at least
3. Define an `BinaryClassificationEvaluator`
4. Using Cross Validation with a 5-fold `CrossValidator`

Questions to answer:
1. Have we improved the ROC-AUC?
2. Which are the average ROC-AUC measurements in the different cross validation runs?
3. Which are the parameters of the best model in the 5 k-fold runs?
4. Which are the metrics of the best model (training) in the 5 k-fold runs? (Use the function above)
5. Which is the ROC-AUC on validation dataset?


#### Adding WeightCols

In [16]:
def col_weigths (dataset):
    data=dataset.toPandas()
    counts, bin_edges = np.histogram(data['loan_status_index'], bins=2)
    proportion_loan_status_index=pd.DataFrame(counts,index=[0,1],columns=['count'])
    proportion_loan_status_index['proportion']=(proportion_loan_status_index['count']/proportion_loan_status_index['count'].sum())
    proportion_loan_status_index
    
    proportion_zeros=proportion_loan_status_index.loc[0,'count']
    proportion_ones=proportion_loan_status_index.loc[1,'proportion']
    
    balancing_ratio=proportion_zeros/dataset.count()
    #print(balancing_ratio,proportion_zeros,dataset.count())
    dataset=dataset.withColumn("classWeights", f.when(dataset.loan_status_index == 1.0,balancing_ratio).otherwise(1-balancing_ratio))
    
    return dataset

In [17]:
trainingData2=col_weigths(trainingData)
validationData2=col_weigths(validationData)

In [18]:
trainingData2.dtypes

[('ID', 'int'),
 ('loan_amnt', 'int'),
 ('term', 'string'),
 ('int_rate', 'float'),
 ('installment', 'double'),
 ('emp_length', 'string'),
 ('annual_inc', 'double'),
 ('purpose', 'string'),
 ('STATE', 'string'),
 ('delinq_2yrs', 'float'),
 ('revol_bal', 'float'),
 ('revol_util', 'float'),
 ('total_pymnt', 'double'),
 ('loan_status', 'int'),
 ('r_ins_minc', 'double'),
 ('term_index', 'double'),
 ('emp_length_index', 'double'),
 ('STATE_index', 'double'),
 ('purpose_index', 'double'),
 ('loan_status_index', 'double'),
 ('term_enc', 'vector'),
 ('emp_length_enc', 'vector'),
 ('purpose_enc', 'vector'),
 ('STATE_enc', 'vector'),
 ('features', 'vector'),
 ('classWeights', 'double')]

In [19]:
lr2= LogisticRegression(featuresCol='features', labelCol="loan_status_index",\
                        weightCol='classWeights')
#1. Grid Search
# Adjust these parameters
lr_paramGrid = ParamGridBuilder() \
                .addGrid(lr2.regParam, [0.01, 0.5]) \
                .addGrid(lr2.elasticNetParam, [0.0,1.0]) \
                .addGrid(lr2.maxIter, [5,10]) \
                .build()

#2. Evaluator
evaluator = BinaryClassificationEvaluator() \
                .setLabelCol("loan_status_index") \
                .setRawPredictionCol("rawPrediction")

#3. Cross-validation: Create 5-fold CrossValidator

lr_cv = CrossValidator(estimator=lr2, estimatorParamMaps=lr_paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations
lr_cvModel = lr_cv.fit(trainingData2)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train=lr_cvModel.transform(trainingData2)
predict_val=lr_cvModel.transform(validationData2)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for validation set after CV  is {}".format(evaluator.evaluate(predict_val)))

# Evaluating CV model
print("AUC: " + str(evaluator.evaluate(predict_train)))
# Means of model accuracy
print("Means of metrics: " + str(lr_cvModel.avgMetrics[0]))

The area under ROC for train set after CV  is 0.9036874131699866
The area under ROC for validation set after CV  is 0.8932894689437012
AUC: 0.9036874131699866
Means of metrics: 0.8704738398509843


### 4. Random Forest Model
1. Define a `ParamGridBuilder` with `maxDepth`, `numTrees` and `maxIter` at least
2. Define an `BinaryClassificationEvaluator` (You can use the above one)
3. Using Cross Validation with a 5-fold `CrossValidator`

Questions to answer:

1. Have we improved the ROC-AUC?
2. Which are the average ROC-AUC measurements in the different cross validation runs?
3. Which are the parameters of the best model in the 5 k-fold runs?
4. Which is the importance of the features?
5. Print full description of model.
6. Which is the ROC-AUC on validation dataset?

In [20]:
#1. Define estimator
rf=RandomForestClassifier(featuresCol='features', labelCol="loan_status_index")

#1. Grid Search
# Adjust these parameters: ######Preguntar maxIter()
rf_paramGrid = ParamGridBuilder() \
                .addGrid(rf.maxDepth, [3,5]) \
                .addGrid(rf.numTrees, [10,15]) \
                .build()

#2. Evaluator
evaluator = BinaryClassificationEvaluator() \
                .setLabelCol("loan_status_index") \
                .setRawPredictionCol("rawPrediction")

#3. Cross-validation: Create 5-fold CrossValidator

rf_cv = CrossValidator(estimator=rf, estimatorParamMaps=rf_paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations
rf_cvModel = rf_cv.fit(trainingData)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train_rf=rf_cvModel.transform(trainingData)
predict_val_rf=rf_cvModel.transform(validationData)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train_rf)))
print("The area under ROC for validation set after CV  is {}".format(evaluator.evaluate(predict_val_rf)))

# Evaluating CV model
print("AUC: " + str(evaluator.evaluate(predict_train_rf)))
# Means of model accuracy
print("Means of metrics: " + str(rf_cvModel.avgMetrics[0]))

The area under ROC for train set after CV  is 0.8170354641855483
The area under ROC for validation set after CV  is 0.7906958078340633
AUC: 0.8170354641855483
Means of metrics: 0.786039138135536


### 5. Gradient Boosting Model
1. Defining a `ParamGridBuilder` with `maxDepth`, `numTrees` and `maxIter` at least (You can use the above one)
2. Define an `BinaryClassificationEvaluator` (You can use the above one)
3. Using Cross Validation with a 5-fold `CrossValidator`

Questions to answer:

1. Have we improved the ROC-AUC?
2. Which are the average ROC-AUC measurements in the different cross validation runs?
3. Which are the parameters of the best model in the 5 k-fold runs?
4. Which is the importance of the features?
5. Print full description of model.
6. Which is the ROC-AUC on validation dataset?

In [21]:
#1. Define estimator
gb=GBTClassifier(featuresCol='features', labelCol="loan_status_index")

#1. Grid Search
# Adjust these parameters: ######Preguntar numTrees()
gb_paramGrid = ParamGridBuilder() \
                .addGrid(gb.maxDepth, [3,5]) \
                .addGrid(gb.maxIter, [10,15]) \
                .build()

#2. Evaluator
evaluator = BinaryClassificationEvaluator() \
                .setLabelCol("loan_status_index") \
                .setRawPredictionCol("rawPrediction")

#3. Cross-validation: Create 5-fold CrossValidator

gb_cv = CrossValidator(estimator=gb, estimatorParamMaps=gb_paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations
gb_cvModel = gb_cv.fit(trainingData)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train_gb=gb_cvModel.transform(trainingData)
predict_val_gb=gb_cvModel.transform(validationData)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train_gb)))
print("The area under ROC for validation set after CV  is {}".format(evaluator.evaluate(predict_val_gb)))

# Evaluating CV model
print("AUC: " + str(evaluator.evaluate(predict_train_gb)))
# Means of model accuracy
print("Means of metrics: " + str(gb_cvModel.avgMetrics[0]))

The area under ROC for train set after CV  is 0.9549841174963601
The area under ROC for validation set after CV  is 0.9369778765806669
AUC: 0.9549841174963601
Means of metrics: 0.910297064092215


### 6. Apply your best model to send the predictions on test

In [22]:
#When we obtain the results from above, we have to decide what model we choose for the final prediction on the testDF1
testData2=col_weigths(testDF1)

bestPipelineModel = lr_cvModel.bestModel
finalpred = bestPipelineModel.transform(testData2)

In [23]:
finalpred.toPandas()

,ID,loan_amnt,term,int_rate,installment,emp_length,annual_inc,purpose,STATE,delinq_2yrs,...,loan_status_index,term_enc,emp_length_enc,purpose_enc,STATE_enc,features,classWeights,rawPrediction,probability,prediction
0,1,5000,36 months,10.600000,162.87,10+ years,24000.0,credit_card,AZ,0.0,...,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5000.0, 162.87, 24000.0, 0.0, 13648.0, 0.8399...",0.0,"[-0.1645910172916846, 0.1645910172916846]","[0.4589448864707543, 0.5410551135292456]",1.0
1,3,2400,36 months,15.900000,84.33,10+ years,12252.0,small_business,IL,0.0,...,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(2400.0, 84.33, 12252.0, 0.0, 2956.0, 0.990000...",0.0,"[-1.305785343639936, 1.305785343639936]","[0.21319296324787138, 0.7868070367521286]",1.0
2,9,5600,60 months,21.200001,152.39,4 years,40000.0,small_business,CA,0.0,...,1.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(5600.0, 152.39, 40000.0, 0.0, 5210.0, 0.33000...",0.0,"[-3.0285202389189005, 3.0285202389189005]","[0.04615392792603417, 0.9538460720739658]",1.0
3,13,9000,36 months,13.400000,305.38,< 1 year,30000.0,debt_consolidation,VA,0.0,...,1.0,"(1.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(9000.0, 305.38, 30000.0, 0.0, 10452.0, 0.9200...",0.0,"[-2.766906330366023, 2.766906330366023]","[0.059138915305775516, 0.9408610846942245]",1.0
4,15,10000,36 months,10.600000,325.74,3 years,100000.0,other,CA,0.0,...,1.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(10000.0, 325.74, 100000.0, 0.0, 11997.0, 0.56...",0.0,"[0.009396757085013883, -0.009396757085013883]","[0.5023491719854756, 0.49765082801452437]",0.0
5,18,3600,36 months,6.030000,109.57,10+ years,110000.0,major_purchase,CT,0.0,...,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3600.0, 109.57, 110000.0, 0.0, 22836.0, 0.159...",0.0,"[1.124350812043257, -1.124350812043257]","[0.7547948553072944, 0.24520514469270555]",0.0
6,26,15000,36 months,9.910000,483.38,2 years,92000.0,credit_card,IL,0.0,...,1.0,"(1.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(15000.0, 483.38, 92000.0, 0.0, 13707.0, 0.939...",0.0,"[1.0660937591981419, -1.0660937591981419]","[0.7438533473390029, 0.2561466526609972]",0.0
7,27,15000,36 months,14.200000,514.64,9 years,60000.0,debt_consolidation,NY,0.0,...,1.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(15000.0, 514.64, 60000.0, 0.0, 5872.0, 0.5799...",0.0,"[-4.104734980809904, 4.104734980809904]","[0.016226739427071722, 0.9837732605729284]",1.0
8,29,4000,36 months,11.700000,132.31,10+ years,106000.0,debt_consolidation,FL,1.0,...,1.0,"(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(4000.0, 132.31, 106000.0, 1.0, 6110.0, 0.3799...",0.0,"[0.755465097447981, -0.755465097447981]","[0.6803683486494366, 0.31963165135056343]",0.0
9,31,4375,36 months,7.510000,136.11,7 years,17108.0,debt_consolidation,NY,0.0,...,1.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [24]:
def export_to_excel(data,name='Lab-15-Assignment3-groupB.csv',header=['ID','prediction'],index=False):
    df=data.toPandas()
    df.to_csv(path_or_buf='./'+name,header=header,columns=['ID','prediction'],index=index)

In [25]:
export_to_excel(finalpred)

In [26]:
sc.stop()